<a href="https://colab.research.google.com/github/wahyunh10/Project-Online-Retail-Dataset/blob/main/Online_Retail_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **RETAIL ONLINE DATASET (EDA + RFM Analysis)**

---



# **Introduction**

Let's Explore Retail Online Dataset and Summarize it using the final dashboard. This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.

In [24]:
#Import libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import re
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# **Import Data**

---


Import the excel file as panda Data Frames and examine it.

In [25]:
#Import the excel file and examine it
df = pd.read_excel('Online Retail.xlsx')
df.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [26]:
df.shape

(541909, 8)

# **Cleansing Dataset**

In [27]:
#Investigate missing value
df.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [28]:
#drop missing value
df = df.dropna()
df.shape

(406829, 8)

In [29]:
#Check the data type
df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [30]:
df.isna().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [31]:
#Cleaning String and data type
df['Description'] = df['Description'].str.replace('.','').str.upper().str.strip()
df['Description'] = df['Description'].replace(r"\s+",' ', regex = True)
df['InvoiceNo'] = df['InvoiceNo'].astype('string')
df['CustomerID'] = df['CustomerID'].astype('string')
df.dtypes

InvoiceNo              string
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID             string
Country                object
dtype: object

In [32]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [33]:
df.describe()

,Quantity,UnitPrice
count,406829.000000,406829.000000
mean,12.061303,3.460471
std,248.693370,69.315162
min,-80995.000000,0.000000
25%,2.000000,1.250000
50%,5.000000,1.950000
75%,12.000000,3.750000
max,80995.000000,38970.000000


This is strange if our dataset contains high negative values. If we look carefully, not all of our dataset contain transactions from selling products activities. Some of fees and costs are included in this dataset. This fees and cost are in StockCode that contain only letter or alphabet (including space) values.

In [34]:
#Cleaning Data Frame where StockCode contains space
df['string'] = df.StockCode.str.isalpha()
df = df[df['string'] != False]
df['space'] = df.StockCode.str.contains(' ')
df = df[df['space'] != False]
df = df.drop(columns=['string', 'space'])
df.shape

(371020, 8)

Next we have to filtering out canceled transaction. In the description of dataset, it is written that there are some canceled transactions presented in dataset. Canceled transactions is indicated by invoice number which starts with letter 'C'.

In [35]:
#Filtering transaction with invoice number started with letter 'C'
df = df[~df.InvoiceNo.str.contains('C')]
df.shape

(363119, 8)

Next, it is time to remove duplicate data.

In [36]:
#Remove ruplicate data in EXACT columns
df.drop_duplicates(inplace = True)
df.shape

(358309, 8)

We move to the next step. In this step, we need to deal with Outlier data. Outlier is a data point in the dataset that differs significantly from the other data or observations. Many statistic procedures are affected by the presence of outliers. So, in this case, removing the outlier may be an option.

In this method, we use IQR (Inter Quartile Range). IQR approach to finding the outliers is the most commonly used and most trusted approach used in the research field.

In [37]:
#Finding the IQR of UnitPrice
percentile25 = df['UnitPrice'].quantile(0.25)
percentile75 = df['UnitPrice'].quantile(0.75)
iqr = percentile75 - percentile25
#Finding upper and lower limit
upper_limit = percentile75 + 1.5 * iqr
lower_limit = percentile25 - 1.5 * iqr
#Removing Outlier from UnitPrice
new_df = df[(df['UnitPrice'] < upper_limit)  & (df['UnitPrice'] > lower_limit)]
print(df.shape)
print(new_df.shape)

(358309, 8)
(326249, 8)


In [38]:
#Finding the IQR of UnitPrice
percentile25 = new_df['Quantity'].quantile(0.25)
percentile75 = new_df['Quantity'].quantile(0.75)
iqr = percentile75 - percentile25
#Finding upper and lower limit
upper_limit = percentile75 + 1.5 * iqr
lower_limit = percentile25 - 1.5 * iqr
#Removing Outlier from UnitPrice
new_df = new_df[(new_df['Quantity'] < upper_limit)  & (new_df['Quantity'] > lower_limit)]
print(new_df.shape)

(303169, 8)


Next it is time to extract the to to make our analysis easier.

In [39]:
#Extract date from datetime column in Pandas
new_df['Date'] = new_df['InvoiceDate'].dt.normalize()
new_df['Year'] = new_df['InvoiceDate'].dt.year
new_df['Month'] = new_df['InvoiceDate'].dt.month_name().str[:3]
new_df['Day'] = new_df['InvoiceDate'].dt.day_name()
new_df['hour'] = new_df['InvoiceDate'].dt.hour
new_df['month_year'] = pd.to_datetime(df['InvoiceDate']).dt.to_period('M')

Finally, for the last step, it's time to create columns Sales to know the total value for each transaction.

In [40]:
new_df['Sales'] = new_df['Quantity'] * new_df['UnitPrice']
new_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Date,Year,Month,Day,hour,month_year,Sales
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,2010,Dec,Wednesday,8,2010-12,20.34
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom,2010-12-01,2010,Dec,Wednesday,8,2010-12,25.50
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom,2010-12-01,2010,Dec,Wednesday,8,2010-12,11.10
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom,2010-12-01,2010,Dec,Wednesday,8,2010-12,11.10
10,536367,22745,POPPY'S PLAYHOUSE BEDROOM,6,2010-12-01 08:34:00,2.10,13047.0,United Kingdom,2010-12-01,2010,Dec,Wednesday,8,2010-12,12.60


# **Exploratory Analysis**

Exploratory data analysis is a way to better understand our data which helps in further Data preprocessing. And data visualization is a key, making the exploratory data analysis process streamline and easily analyzing data using plots and charts.

# **Sales vs Quantity**
**Sales and Quantity Based on Country**

In [43]:
Sales_Country = new_df.groupby(['Country']).agg({'InvoiceNo': ['nunique'],
                                      'InvoiceNo': 'count', 'Sales':'sum', 'Quantity':'sum'}).rename(columns={'InvoiceNo':'Total_Order'}).sort_values('Sales', ascending=False)
print(Sales_Country.head(10))

                Total_Order       Sales  Quantity
Country                                          
United Kingdom       272708  3352647.62   1970315
Germany                6934   126673.74     78420
France                 6535   115502.90     72293
EIRE                   4920    95928.40     53805
Spain                  1849    27338.06     16945
Switzerland            1313    26943.79     16635
Belgium                1527    25941.26     16325
Portugal               1176    18982.35     11214
Norway                  717    16711.56      8772
Netherlands             576    13980.84      6703
